In [19]:
from transformers import *
import tensorflow as tf 
from tensorflow.keras.layers import * 
from tensorflow.keras.models import * 
from tensorflow.keras.callbacks import *
from tensorflow.keras.preprocessing.text import *  
from tensorflow.keras.losses import * 
from tensorflow.keras.optimizers import *
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import random 
import re 
import math 
from tqdm import tqdm 
import sklearn

import torch 
import torch.nn as nn 
import torch.functional as f
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
import time
import datetime


In [20]:
train = pd.read_csv("drive/MyDrive/dacon_NLP/nlp_train.csv") 
test = pd.read_csv("drive/MyDrive/dacon_NLP/nlp_test.csv") 

train.shape, test.shape, train['label'].nunique()

((174304, 13), (43576, 12), 46)

In [21]:
num_classes = train['label'].nunique() # 46 

In [22]:
submission = pd.read_csv("drive/MyDrive/dacon_NLP/sample_submission.csv") 

In [23]:
# clean unnecessary symbols
def clean_text(sent):
    sent_clean=re.sub("[^가-힣ㄱ-하-ㅣ]", " ", sent)
    return sent_clean

In [24]:
def split_text(s, overlap = 20, chunk_size = 50): 
    total = [] 
    partial = [] 
    if len(s.split()) // (chunk_size - overlap) > 0:  
        n = len(s.split()) // (chunk_size - overlap) 
    else: 
        n = 1 
    for w in range(n): 
        if w == 0: 
            partial = s.split()[:chunk_size] 
            total.append(" ".join(partial)) 
        else:  
            partial = s.split()[w*(chunk_size - overlap):w*(chunk_size - overlap) + chunk_size]
            total.append(" ".join(partial)) 
    return total

In [25]:
train['요약문_내용'] = train['요약문_연구목표'] + train['요약문_연구내용'] + train['요약문_기대효과'] 
test['요약문_내용'] = test['요약문_연구목표'] + test['요약문_연구내용'] + test['요약문_기대효과'] 

In [26]:
train['요약문_내용'].fillna('NAN',inplace=True) 
test['요약문_내용'].fillna('NAN',inplace=True)

In [27]:
train['사업명'].fillna('NAN',inplace=True) 
train['사업_부처명'].fillna('NAN',inplace=True) 
train['내역사업명'].fillna('NAN',inplace=True) 
train['과제명'].fillna('NAN',inplace=True) 
train['요약문_한글키워드'].fillna('NAN',inplace=True) 
train['요약문_영문키워드'].fillna('NAN',inplace=True)

In [11]:
checkpoint = torch.load('./drive/MyDrive/dacon_NLP/XLM_ROBERTA_BASE_voting_6')
test_model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=46) 
test_model.load_state_dict(checkpoint) 
test_model.cuda() 

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (La

In [12]:
import torch.nn.functional as nnf

test['사업명'].fillna('NAN',inplace=True) 
test['사업_부처명'].fillna('NAN',inplace=True) 
test['내역사업명'].fillna('NAN',inplace=True) 
test['과제명'].fillna('NAN',inplace=True) 
test['요약문_한글키워드'].fillna('NAN',inplace=True)
test['요약문_영문키워드'].fillna('NAN',inplace=True)

In [13]:
BATCH_SIZE = 32
NUM_EPOCHS = 30
VALID_SPLIT = 0.1 
MAX_LEN = 512 # max token size for BERT, ELECTRA

device = torch.device("cuda")

In [14]:
## Now we tokenize each data and make sure they all lie within the 512 tokenization range 
## if not check how many have token length greater than 512 

tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base") 

def roberta_tokenizer(sent, MAX_LEN):  
    encoded_dict = tokenizer.encode_plus(
        text = sent, 
        add_special_tokens = True, # add [CLS] and [SEP]
        pad_to_max_length = False, 
        return_attention_mask = True # constructing attention_masks 
    )  
    
    input_id = encoded_dict['input_ids'] 
    attention_mask = encoded_dict['attention_mask'] # differentiate padding from non padding 
    
    if len(input_id) > 512: # head + tail methodology 
        input_id = input_id[:129] + input_id[-383:] 
        attention_mask = attention_mask[:129] + attention_mask[-383:]  
        print("Long Text!! Using Head+Tail Truncation")
    elif len(input_id) <= 512: 
        input_id = input_id + [0]*(512 - len(input_id)) 
        attention_mask = attention_mask + [0]*(512 - len(attention_mask))
        
    return input_id, attention_mask 


In [16]:
test_contents = test['요약문_내용'].values 
test_feature1 = test['사업명'].values 
test_feature2 = test['사업_부처명'].values 
test_feature3 = test['내역사업명'].values 
test_feature4 = test['과제명'].values 
test_feature5 = test['요약문_한글키워드'].values 
test_feature6 = test['요약문_영문키워드'].values 


predicted_classes = []

# change to eval mode 
test_model.eval() 


for i in tqdm(range(test_contents.shape[0]), position=0, leave=True):
    f1 = clean_text(str(test_feature1[i])) 
    f2 = clean_text(str(test_feature2[i])) 
    f3 = clean_text(str(test_feature3[i])) 
    f4 = clean_text(str(test_feature4[i])) 
    f5 = str(test_feature5[i])  
    f6 = str(test_feature6[i]) 
    splitted = split_text(clean_text(str(test_contents[i]))) 
    # make predictions for each splitted text 
    probabilities = [] 
    for text in splitted: 
        test_text = f1 + " " + f2 + " " + f3 + " " + f4 + " " + f5 + " " + f6 + " " + text 
        # tokenize test text 
        input_id, attention_mask = roberta_tokenizer(test_text, MAX_LEN=MAX_LEN) 
        input_id = torch.tensor(input_id)
        attention_mask = torch.tensor(attention_mask) 
        # reshape into (batch, MAX_LEN)
        input_id = torch.reshape(input_id, (-1,MAX_LEN)) 
        attention_mask = torch.reshape(attention_mask, (-1,MAX_LEN)) 
        # move tensor to cuda 
        input_id = input_id.to(device) 
        attention_mask = attention_mask.to(device) 
        
        with torch.no_grad(): 
            outputs = test_model(input_id, 
                                 token_type_ids=None, 
                                 attention_mask=attention_mask) 
        
        logits = outputs[0]
        
        # obtain softmax probabilities 
        prob = nnf.softmax(logits, dim=1).flatten()
        probabilities.append(prob)
    
    # soft voting 
    prob_sum = np.zeros(46) 
    for i in range(len(probabilities)): 
        for j in range(46): 
            prob_sum[j] += probabilities[i][j] 
            
    
    prob_sum /= len(probabilities)  
    
    predicted_class = np.argmax(prob_sum)
    
    predicted_classes.append(predicted_class)


  7%|▋         | 3141/43576 [11:13<2:22:16,  4.74it/s]

Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation


 20%|█▉        | 8594/43576 [30:54<2:21:53,  4.11it/s]

Long Text!! Using Head+Tail Truncation


 62%|██████▏   | 27182/43576 [1:36:50<47:45,  5.72it/s]

Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation


100%|██████████| 43576/43576 [2:35:31<00:00,  4.67it/s]


In [17]:
submission.iloc[:,1] = predicted_classes 

submission


,index,label
0,174304,0
1,174305,0
2,174306,0
3,174307,0
4,174308,0
...,...,...
43571,217875,0
43572,217876,0
43573,217877,2
43574,217878,0


In [18]:
submission.to_csv("drive/MyDrive/dacon_NLP/XLM_ROBERTA_voting.csv",index=False)

# Ensemble 

Try a soft voting ensemble from Electra and XLM Roberta 

In [29]:
## load electra  
electra_checkpoint = torch.load("./drive/MyDrive/dacon_NLP/ELECTRA_MORE_FEATURES_6")
test_electra = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=46)
test_electra.load_state_dict(electra_checkpoint) 
test_electra.cuda()

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [30]:
## load xlm-roberta 
xlm_roberta_checkpoint = torch.load("./drive/MyDrive/dacon_NLP/XLM_ROBERTA_BASE_voting_6")
test_roberta = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=46) 
test_roberta.load_state_dict(xlm_roberta_checkpoint) 
test_roberta.cuda() 

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (La

In [34]:
# define Ko-ELECTRA tokenizer 
tokenizer_electra = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator") 

def electra_tokenizer(sent, MAX_LEN):  
    encoded_dict = tokenizer_electra.encode_plus(
        text = sent, 
        add_special_tokens = True, # add [CLS] and [SEP]
        pad_to_max_length = False, 
        return_attention_mask = True # constructing attention_masks 
    )  
    
    input_id = encoded_dict['input_ids'] 
    attention_mask = encoded_dict['attention_mask'] # differentiate padding from non padding 
    token_type_id = encoded_dict['token_type_ids'] # differentiate two sentences, not "really" necessary for now    
    
    if len(input_id) > 512: # head + tail methodology 
        input_id = input_id[:129] + input_id[-383:] 
        attention_mask = attention_mask[:129] + attention_mask[-383:]  
        token_type_id = token_type_id[:129] + token_type_id[-383:]    
        print("Long Text!! Using Head+Tail Truncation")
    elif len(input_id) <= 512: 
        input_id = input_id + [0]*(512 - len(input_id)) 
        attention_mask = attention_mask + [0]*(512 - len(attention_mask))
        token_type_id = token_type_id + [0]*(512 - len(token_type_id))  
        
    return input_id, attention_mask, token_type_id


In [32]:
# define XLM-RoBERTa tokenizer
## Now we tokenize each data and make sure they all lie within the 512 tokenization range 
## if not check how many have token length greater than 512 

tokenizer_roberta = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base") 

def roberta_tokenizer(sent, MAX_LEN):  
    encoded_dict = tokenizer_roberta.encode_plus(
        text = sent, 
        add_special_tokens = True, # add [CLS] and [SEP]
        pad_to_max_length = False, 
        return_attention_mask = True # constructing attention_masks 
    )  
    
    input_id = encoded_dict['input_ids'] 
    attention_mask = encoded_dict['attention_mask'] # differentiate padding from non padding 
    
    if len(input_id) > 512: # head + tail methodology 
        input_id = input_id[:129] + input_id[-383:] 
        attention_mask = attention_mask[:129] + attention_mask[-383:]  
        print("Long Text!! Using Head+Tail Truncation")
    elif len(input_id) <= 512: 
        input_id = input_id + [0]*(512 - len(input_id)) 
        attention_mask = attention_mask + [0]*(512 - len(attention_mask))
        
    return input_id, attention_mask 


In [39]:
test_contents = test['요약문_내용'].values 
test_feature1 = test['사업명'].values 
test_feature2 = test['사업_부처명'].values 
test_feature3 = test['내역사업명'].values 
test_feature4 = test['과제명'].values 
test_feature5 = test['요약문_한글키워드'].values 
test_feature6 = test['요약문_영문키워드'].values 


predicted_classes = []

# change to eval mode 
test_electra.eval() 
test_roberta.eval() 


for i in tqdm(range(test_contents.shape[0]), position=0, leave=True):
    f1 = clean_text(str(test_feature1[i])) 
    f2 = clean_text(str(test_feature2[i])) 
    f3 = clean_text(str(test_feature3[i])) 
    f4 = clean_text(str(test_feature4[i])) 
    f5 = str(test_feature5[i])  
    f6 = str(test_feature6[i]) 
    splitted = split_text(clean_text(str(test_contents[i]))) 
    # make predictions for each splitted text 
    probabilities_electra = [] # store probabilities for electra 
    probabilities_roberta = [] # store probabilities for xlm-roberta 
    for text in splitted: 
        test_text = f1 + " " + f2 + " " + f3 + " " + f4 + " " + f5 + " " + f6 + " " + text 
        ''' XLM-RoBERTa predictions '''
        # tokenize test text 
        input_id, attention_mask = roberta_tokenizer(test_text, MAX_LEN=MAX_LEN) 
        input_id = torch.tensor(input_id)
        attention_mask = torch.tensor(attention_mask) 
        # reshape into (batch, MAX_LEN)
        input_id = torch.reshape(input_id, (-1,MAX_LEN)) 
        attention_mask = torch.reshape(attention_mask, (-1,MAX_LEN)) 
        # move tensor to cuda 
        input_id = input_id.to(device) 
        attention_mask = attention_mask.to(device) 
        
        with torch.no_grad(): 
            outputs = test_roberta(input_id, 
                                   token_type_ids=None, 
                                   attention_mask=attention_mask) 
        
        logits = outputs[0]
        
        # obtain softmax probabilities 
        prob = nnf.softmax(logits, dim=1).flatten()
        probabilities_roberta.append(prob)

        ''' KoELECTRA predictions ''' 
        # tokenize test text 
        input_id, attention_mask, token_type_ids = electra_tokenizer(test_text, MAX_LEN=MAX_LEN) 
        input_id = torch.tensor(input_id)
        attention_mask = torch.tensor(attention_mask) 
        token_type_ids = torch.tensor(token_type_ids) 
        # reshape into (batch, MAX_LEN)
        input_id = torch.reshape(input_id, (-1, MAX_LEN)) 
        attention_mask = torch.reshape(attention_mask, (-1, MAX_LEN)) 
        token_type_ids = torch.reshape(token_type_ids, (-1, MAX_LEN)) 
        # move tensor to cuda 
        input_id = input_id.to(device) 
        attention_mask = attention_mask.to(device) 
        token_type_ids = token_type_ids.to(device) 
        with torch.no_grad(): 
            outputs = test_electra(input_id, 
                                   token_type_ids=token_type_ids,
                                   attention_mask = attention_mask)

        logits = outputs[0]
        
        # obtain softmax probabilities  
        prob = nnf.softmax(logits, dim=1).flatten() 
        probabilities_electra.append(prob) 
    
    # soft voting 
    prob_sum = np.zeros(46) 
    for i in range(len(probabilities_roberta)): 
        for j in range(46): 
            prob_sum[j] += probabilities_roberta[i][j] 
    for i in range(len(probabilities_electra)):  
        for j in range(46): 
            prob_sum[j] += probabilities_electra[i][j]  
            
    
    prob_sum /= (2 * len(probabilities)) 
    
    predicted_class = np.argmax(prob_sum)
    
    predicted_classes.append(predicted_class)


  7%|▋         | 3140/43576 [22:20<5:24:53,  2.07it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (541 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (565 > 512). Running this sequence through the model will result in indexing errors


Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation


  7%|▋         | 3141/43576 [22:20<4:42:47,  2.38it/s]

Long Text!! Using Head+Tail Truncation


 20%|█▉        | 8594/43576 [1:01:26<4:48:05,  2.02it/s]

...

 80%|███████▉  | 34724/43576 [4:05:08<1:04:05,  2.30it/s]

Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation


 92%|█████████▏| 40015/43576 [4:43:23<16:35,  3.58it/s]

Long Text!! Using Head+Tail Truncation


100%|██████████| 43576/43576 [5:08:45<00:00,  2.35it/s]


In [40]:
submission.iloc[:,1] = predicted_classes 
submission

,index,label
0,174304,0
1,174305,0
2,174306,0
3,174307,0
4,174308,0
...,...,...
43571,217875,0
43572,217876,0
43573,217877,2
43574,217878,0


In [41]:
submission.to_csv("drive/MyDrive/dacon_NLP/ELECTRA_XLM_ROBERTA_soft_ensemble.csv",index=False)